In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
from datetime import timedelta
import re
import shutil

import glob
%matplotlib inline

import matplotlib as mpl
mpl.use('agg')

# import personal modules
from plotter import plot_mclimate_forecast
import mclimate_funcs as mclim_func
from build_html_table import create_html_table

In [2]:
%%time

######################
### VARS TO UPDATE ###
######################
fdate = None ## initialization date in YYYYMMDD format
model = 'GEFS' ## 'GEFSv12_reforecast', 'GFS', 'GEFS', 'GEFS_archive'
map_ext = [-170., -120., 40., 65.] ## map extent [minlon, maxlon, minlat, maxlat]
table_ext = [-141., -130., 54.5, 60.] ## extent to choose the maximum value from for the table [minlon, maxlon, minlat, maxlat]
fig_path = 'figs/images_operational/'
os.makedirs(os.path.dirname(fig_path), exist_ok=True)

###########
### IVT ###
###########
varname = 'ivt' ## 'freezing_level' or 'ivt'
forecast, ds = mclim_func.run_compare_mclimate_forecast(varname, fdate, model)
step_lst = ds.step.values

for i, step in enumerate(step_lst):
    print(step)
    out_fname = fig_path + '{0}_mclimate_F{1}'.format(varname, step)
    plot_mclimate_forecast(ds, forecast, step=step, varname='ivt', fname=out_fname, ext=map_ext)

######################
### FREEZING LEVEL ###
######################
varname = 'freezing_level'
forecast, ds1 = mclim_func.run_compare_mclimate_forecast(varname, fdate, model)

for i, step in enumerate(step_lst):
    print(step)
    out_fname = fig_path + '{0}_mclimate_F{1}'.format(varname, step)
    plot_mclimate_forecast(ds1, forecast, step=step, varname='freezing_level', fname=out_fname, ext=map_ext)

## put into single dataset for table
ds = ds.rename({'mclimate': 'IVT'})
ds1 = ds1.rename({'mclimate': 'freezing_level'})
ds2 = xr.merge([ds, ds1])
ds2 = ds2.sortby('lat')

###################
### BUILD TABLE ###
###################
df = create_html_table(ds2, table_ext)
df

/home/dnash/miniconda3/envs/SEAK-impacts/lib/python3.12/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gini' loading failed:
Struct() takes at most 1 argument (3 given)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


08 15
6.0
12.0
18.0
24.0
30.0
36.0
42.0
48.0
54.0
60.0
66.0
72.0
78.0
84.0
90.0
96.0
102.0
108.0
114.0
120.0
126.0
132.0
138.0
144.0
150.0
156.0
162.0
168.0
CPU times: user 42.9 s, sys: 16.6 s, total: 59.5 s
Wall time: 31.3 s


In [5]:
## write to text file
html = df.to_html(index=False, formatters={'Hour': lambda x: '<b>' + x + '</b>'}, escape=False)

# write html to file
text_file = open("out/table.html", "w")
text_file.write(html)
text_file.close()